# CCTV 설치 입지 제안 프로젝트
**추출된 데이터에 대한 규정 및 데이터 통합 과정**

0. 모든 데이터에는 좌표와 데이터 프레임 정보가 기입되어 있어야 한다.
1. 인프라 정보는 infra_inf와 같은 정보가 담겨 있어야 한다.
2. 이후, cctv(CCTV설치정보),light(가로등 설치정보),illigal(불법주정차 단속정보),park(공영주차장 설치정보) 데이터를 추출한다.
3. 인프라 데이터의 좌표 정보를 기준으로 클러스터링을 진행한다.
4. 이후 인프라 데이터의 좌표 정보를 기반으로 다른 데이터 프레임도 클러스터링을 진행한다.
5. 클러스터링 번호를 기준으로 모든 데이터를 병합한다.
6. 단속건수의 이상치를 제거한다.

-----
**대시보드의 역할**
1. 병합한 데이터를 기준으로 불법 주정차를 유발하는 인프라 정보(통합된)에 대해 분석한다. 
2. cctv가 설치 되었을 때 가장 효과가 좋은 (불법 주정차 단속 건수가 가장 많이 증가하는 클러스터 정보) 지역을 제안한다.

**사용자의 역할**
1. 대시보드에서 확인한 분석 정보를 바탕으로 인사이트를 확인하고 대상 지역의 특성과 비교한다.
2. 대상 지역에 대한 모델의 제안을 기반으로 가장 효과적일 것으로 예상되는 세부 설치 지역을 탐색한다.

---
**입력 변수**
- cluster_value = 지역 세분화 관련 변수 (클러스터링 파라미터)

**데이터 프레임 정보**
- infra_df = 인프라 정보
- cctv = CCTV 설치 정보
- light = 가로등 설치 정보
- park = 공영 주차장 설치 정보
- illigal = 불법 주정차 단속 정보

---


In [383]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'NanumGothic'

In [384]:
infra_df = pd.read_csv('인프라.csv')
infra_df['정보'] = infra_df['정보'].str.replace('서울시광진구','',regex=False)
infra_df['정보'] = infra_df['정보'].str.replace('인허가정보.csv','',regex=False)
infra_inf= infra_df['정보'].value_counts().index
infra_df = infra_df.rename(columns={'좌표정보(X)':'경도','좌표정보(Y)':'위도'})

In [385]:
infra_inf
# 업장 종류

Index(['일반음식점', '휴게음식점', '미용업', '의원', '세탁업', '안전상비의약품판매업소', '노래연습장업', '당구장업',
       '제과점영업', '약국', '단란주점영업', '집단급식소', '숙박업', '목욕장업', '체육도장업', '골프연습장업',
       '동물병원', '집단급식소식품판매업', '영화상영관', '외국인관광도시민박업', '민방위대피시설', '공연장', '대규모점포',
       '유흥주점영업', '병원', '관광숙박업', '수영장업'],
      dtype='object')

In [386]:
cctv=pd.read_csv('cctv장소.csv')
cctv['정보'] = 'cctv'
cctv = cctv[['위도','경도','정보']]
park = pd.read_csv('공영주차장.csv')
park['정보']='공영주차장'
park = park[['위도','경도','정보']]
light = pd.read_csv('가로등.csv')
light['정보']= '가로등'
light = light[['위도','경도','정보']]
illigal = pd.read_csv('불법주정차단속.csv')
illigal['정보'] = '단속'
illigal = illigal[['위도','경도','정보']]

In [387]:
cluster_value = 100
# 입력 변수

In [388]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=cluster_value, random_state=42)
infra_df = infra_df.iloc[infra_df[['위도','경도']].dropna().index]
infra_df['cluster'] = kmeans.fit_predict(infra_df[['위도','경도']].dropna())
def clustering(df):
    df= df.iloc[df[['위도','경도']].dropna().index]
    df['cluster']=kmeans.predict(df[['위도','경도']].dropna())
    return df
cctv = clustering(cctv)
light = clustering(light)
park = clustering(park)
illigal = clustering(illigal)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\HP\AppData\Local\Temp\ipykernel_2800\2723768720.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster']=kmeans.predict(df[['위도','경도']].dropna())
C:\Users\HP\AppData\Local\Temp\ipykernel_2800\2723768720.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cl

In [389]:
pivot=pd.pivot_table(data=infra_df,index='cluster',columns='정보',aggfunc='count',fill_value=0)
pivot = pivot['경도']
for i in [cctv,light,park,illigal]:
    t=pd.pivot_table(data=i,index='cluster',columns='정보',aggfunc='count',fill_value=0)['경도']
    pivot = pd.concat([pivot,t],axis=1)
    pivot = pivot.fillna(0)    

---

**4분위수를 활용한 이상치 제거**

In [393]:
data = pivot['단속']
q1,q3 = np.percentile(data,[25,75])
iqr = q3-q1
lower_bound = q1-(iqr*1.5)
upper_bound = q3+(iqr*1.5)
pivot = pivot[(pivot['단속']>=lower_bound)&(pivot['단속']<=upper_bound)]

---

# 모델링 및 학습

**재표본추출을 통한 학습 데이터 생성**
- 기존에 만든 pivot 데이터프레임은 광진구 전체를 1000개로 클러스터링 한 모집단의 성격을 띠고 있다.
- 학습 데이터프레임의 변동성을 반영하기 위해 sample 메서드를 활용해 30개씩 1000번 추출한다.
- 데이터 프레임을 앙상블 기법으로 학습한다.
- 학습 평가 지표는 MAE 로 설정하고 학습 목표는 MAE 값의 2 이하 달성이다.

---
**모델의 역할**
- 모델은 입력받은 메타 데이터프레임의 클러스터에 CCTV 개수를 1개씩 더하고 단속 건수의 변화를 확인한다.
- 변화 폭이 가장 큰 지역에 CCTV를 설치할 것을 제안한다.
- 이외에도 입력받은 메타 데이터프레임의 분석 정보를 제공함으로써 사용자의 데이터 기반 의사결정을 보조한다.

In [395]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,KFold, cross_val_score

**재표본추출**

In [397]:
agu_df = pivot.copy()
for i in range(100):
    t = pivot.sample(30)
    agu_df = pd.concat([agu_df,t])

In [398]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
agu_df_x = agu_df.copy()
agu_df_x = agu_df_x.drop('단속',axis=1)
agu_df_s = scaler.fit_transform(agu_df_x)
agu_df_x.loc[:] = agu_df_s


**모델 교차검증 및 학습**

In [399]:
from xgboost import XGBRegressor
x = agu_df_x
y = agu_df['단속']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2)
model = XGBRegressor(learning_rate=0.1,n_estimators=100,max_depth=8)
kf = KFold(n_splits=10)
scores = cross_val_score(model, x_train, y_train, cv=kf,scoring='neg_mean_absolute_error')
scores*-1

array([0.02410421, 0.02455967, 0.02261103, 0.02028858, 0.02115664,
       0.0240203 , 0.02294222, 0.02422751, 0.02652981, 0.02181683])

In [400]:
model.fit(x_train,y_train)
pred = model.predict(x_test)

In [401]:
print('mae:',mean_absolute_error(y_test,pred))
print('mse:',mean_squared_error(y_test,pred))
print('r2:',r2_score(y_test,pred))

mae: 0.02266157344345329
mse: 0.0009549188892484818
r2: 0.999999998197165


In [402]:
import pickle
with open('new_model.pkl', 'wb') as file:
    pickle.dump(model, file)

---

# 최적의 설치 클러스터 확인
**pivot 데이터 프레임에 대해 cctv를 추가한 경우의 단속건수 변화 확인**
- cctv 변화 전, 후에 따른 예측 값 도출
- cctv 변화 전 예측 값과 원본 값의 차이가 작은 데이터에 대해 변화 전,후 예측값의 차이 중 가장 큰 값을 도출

In [403]:
import pickle
with open('new_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [404]:
test = pivot.copy()
modify_cctv_values = 1
# 클러스터 당 변화 시킬 CCTV 대수

In [405]:
test_pred = test.drop('단속',axis=1)
pred_0 = model.predict(test_pred)
test_pred['cctv'] = test_pred['cctv']+modify_cctv_values
pred_= model.predict(test_pred) + np.mean(scores)
result = pd.DataFrame([pred_,pred_0,test['단속']]).T
result = result.dropna()
result.columns = ['변화CCTV_예측','원본CCTV_예측','원본']
result['예측값차이_절댓값'] = np.abs(result['원본']-result['원본CCTV_예측'])
result['예측값간차이'] = result['변화CCTV_예측']-result['원본CCTV_예측']

max_eff_cluster = result.sort_values(by='예측값차이_절댓값').index[0]
effect_ = result['예측값간차이'][max_eff_cluster]

In [406]:
print('설치 시 가장 큰 효과를 볼 수 있는 클러스터 값 : ',max_eff_cluster)
print('설치 효과 : ',effect_)
print('클러스터 당 CCTV 추가설치 대수 : ',modify_cctv_values)

설치 시 가장 큰 효과를 볼 수 있는 클러스터 값 :  18
설치 효과 :  2079.53067779541
클러스터 당 CCTV 추가설치 대수 :  1


# 구체적 위치 시각화 및 인사이트 제공

In [408]:
max_effect_infra_df = infra_df[infra_df['cluster']==max_eff_cluster]

**구체적인 클러스터 위치 지도 시각화**

In [409]:
import folium
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=13)
for idx, row in max_effect_infra_df.iterrows():
    folium.Marker([row['위도'], row['경도']], popup=row['정보']).add_to(seoul_map)

In [410]:
seoul_map

---

**모델 변수 중요도 관련 인사이트 제공**

In [411]:
feature_importance = pd.DataFrame(data=model.feature_importances_,index=x_train.columns)*100
feature_importance = feature_importance.sort_values(by=0,ascending=False)
over_5_feature_importance = feature_importance[feature_importance[0]>=5]

In [412]:
print('모델에 5% 이상 영향을 미친 인프라 정보 : ',[i for i in over_5_feature_importance.index])

모델에 5% 이상 영향을 미친 인프라 정보 :  ['cctv', '일반음식점', '가로등', '약국', '세탁업']


---

**종속변수 분류를 통한 인사이트 제공**

In [413]:
bins = [-float('inf'),q1,q3,q3*1.5,float('inf')]
labels = ['under_75%','over_low','over_mid','over_high']
agu_df_insight = agu_df.copy()
agu_df_insight['단속수준'] = pd.cut(agu_df['단속'],bins=bins,labels=labels)

In [414]:
agu_df_insight.groupby(by='단속수준').mean()

정보,골프연습장업,공연장,관광숙박업,노래연습장업,단란주점영업,당구장업,대규모점포,동물병원,목욕장업,미용업,...,일반음식점,제과점영업,집단급식소,집단급식소식품판매업,체육도장업,휴게음식점,cctv,가로등,공영주차장,단속
단속수준,,,,,,,,,,,,,,,,,,,,,
under_75%,0.262615,0.223624,0.188073,4.000000,3.330275,5.018349,0.326835,0.463303,1.263761,31.407110,...,169.457569,4.151376,1.635321,0.360092,1.065367,34.549312,0.103211,20.839450,0.253440,53.098624
over_low,0.820722,0.206054,0.171129,4.182189,2.817229,4.884168,0.289290,0.621071,1.406286,31.949942,...,139.540163,3.061118,3.068685,0.574505,1.335274,36.242142,0.511059,26.362049,0.196158,477.484866
over_mid,1.719457,0.144796,0.199095,6.529412,7.185520,4.131222,0.000000,0.669683,1.524887,26.212670,...,119.416290,3.452489,3.135747,0.832579,0.841629,35.778281,1.000000,34.429864,1.108597,1637.986425
over_high,0.690647,0.219424,0.000000,8.924460,3.496403,6.661871,0.356115,1.679856,0.856115,42.971223,...,201.866906,3.852518,0.931655,1.147482,2.345324,42.744604,1.474820,33.111511,0.302158,2259.863309


In [ ]:
!streamlit run streamlit_project.py

---